In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import time
from pprint import pprint
import requests
from datetime import date, timedelta, datetime
import json
# Import API key
from config import weather_api_key
from pprint import pprint

### Get Long Lat list of tuples to pass into Weather API

In [ ]:
geojson_data = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/4.5_month.geojson'

In [ ]:
response = requests.get(geojson_data).json()
features = response["features"]
test = features[0]['properties']
coords_list = []
for data in features:
    time = data['properties']['time']
    magnitude = data['properties']['mag']
    place = data['properties']['place']
    
    long_coords = data["geometry"]["coordinates"][0]
    lat_coords = data["geometry"]["coordinates"][1]
    
    coords = {
            'latlng': f"{lat_coords}, {long_coords}",
            'lat': lat_coords,
            'long': long_coords,
            'time': time,
            'magnitude': magnitude,
            'place': place}
    
    coords_list.append(coords)
pprint(coords_list)
# coords_list

In [ ]:
len(coords_list)

-----

### Use Coordinates to get historical weather data.
Coordinates represent earthquake lat long location

This section creates the table WeatherSeries in the database. WeatherSeries has 4 day weather forecast and matching earthquake data.

In [ ]:
from tqdm import tqdm
from tqdm import tqdm_notebook

In [ ]:
# for coordinates in coords_list:
#     # Lat / long
#     coords = coordinates['latlng']
# coords

In [ ]:
# base_url = 'http://api.weatherstack.com/historical'
# params_weather = {'access_key': weather_api_key, 
#                 'query': coords,
#                 'historical_date_start': time_range_begin,
#                 'historical_date_end': time_range_end,
#                 'units': 'f'
#              }
# response = requests.get(base_url, params=params_weather).url
# response

In [ ]:
pbar = tqdm(total=len(coords_list))

tuple_of_data = []
add_tuple = {}
counter = 0

def checkForTwo(number):
#     if (int(number)) <= 1:
#         number = '1'
    if (int(number)) < 10:
        number = "0" + str(number)
    else:
#         print(number)
        number = str(number)
        pass
    return number

for coordinates in coords_list:
    # Lat / long
    coords = coordinates['latlng']
    
    ######################
    ## Earthquake time conversion
    ######################
    earthquake_time = coordinates['time']
    dt3 = datetime.fromtimestamp(earthquake_time / 1000)
    # Time range end is the day of the earthquake
    time_range_end = str(dt3.year) + "-" + checkForTwo(dt3.month) + "-" + checkForTwo(dt3.day)
    
    # Time range begin is 3 days before the earthquake
    four_day_series = dt3 - timedelta(3)
    four_day_day = four_day_series
    time_range_begin = str(dt3.year) + "-" + checkForTwo(dt3.month) + "-" + checkForTwo(four_day_day.day)
    ######################
    
    base_url = 'http://api.weatherstack.com/historical'
    params_weather = {'access_key': weather_api_key, 
                    'query': coords,
                    'historical_date_start': time_range_begin,
                    'historical_date_end': time_range_end,
                    'units': 'f'
                 }
    
    response = requests.get(base_url, params=params_weather).json()
    # url_link = requests.get(base_url, params=params_weather).url

    try: 
        # extract results
        location = response.get('location')
        # City, Country, Region
        city = location["name"]
        country = location["country"]
        region = location["region"]

        # Historical Only
        historical = response['historical']
#         print(region)
        for hist in historical:
            add_tuple = {
                'city': city,
                'country': country,
                'region': region,
                'avgtemp': historical[hist]['avgtemp'],
                'date': historical[hist]['date'],
                'date_epoch': historical[hist]['date_epoch'],
                'maxtemp': historical[hist]['maxtemp'],
                'mintemp': historical[hist]['mintemp'],
                'sunhour': historical[hist]['sunhour'],
                'totalsnow': historical[hist]['totalsnow'],
                'uv_index': historical[hist]['uv_index'],
                'long': coordinates['long'],
                'lat': coordinates['lat'],
                'time': coordinates['time'],
                'magnitude': coordinates['magnitude'],
                'place': coordinates['place']
            }
            tuple_of_data.append(add_tuple)
            pbar.update(1)
    except TypeError as e:
        pbar.update(1)
    # print(e)
        continue
    pbar.update(1)
    counter += 1

pbar.close() 

# tuple_of_data

In [ ]:
len(tuple_of_data)

## Saving to JSON

In [ ]:
from datetime import datetime
now = datetime.now()

In [ ]:
# Get time
timedate = now.strftime("%Y-%m-%d_%H_%M_%S")

# SAVE: Player_position
with open(f'../{timedate}_4_day_weather_for_earthquakes.json', 'w') as fp:
    json.dump(tuple_of_data, fp)
    

In [ ]:
# del tuple_of_data

## Opening JSON .. Do not have to run the api again.

In [ ]:
# Open raw json to parse
with open('../2020-08-13_12_52_41_4_day_weather_for_earthquakes.json', 'r') as f:
    weather_earthquake_data = json.load(f)

In [ ]:
# Dataframe to see raw(cleaned) data
all_logs = pd.DataFrame(weather_earthquake_data)
all_logs.head()

## Upload to PostgreSQL

In [ ]:
# Use this to clear out the db
# ----------------------------------
# # Session.rollback(self)
# Base.metadata.drop_all(engine)
# session.commit()

In [ ]:
# Delete previous entries
# del WeatherSeries
# del LatLong

In [3]:
# Dependencies
# ----------------------------------
# Imports the method used for connecting to DBs
from sqlalchemy import create_engine
# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Text, DateTime, Float, Boolean, ForeignKey
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session

In [4]:
# Create Weather and Earthquake Classes
# ----------------------------------
# Sets an object to utilize the default declarative base in SQL Alchemy
Base = declarative_base()

In [ ]:
## Class base template to upload to sqlite
class WeatherSeries(Base):
    __tablename__ = 'weatherSeries'

    id = Column(Integer, primary_key=True)
    city = Column(String(50))
    country = Column(String(200))
    region = Column(String(80))
    avgtemp = Column(Float)
    date = Column(String(12))
    date_epoch = Column(Float)
    maxtemp = Column(Float)
    mintemp = Column(Float)
    sunhour = Column(Float)
    totalsnow = Column(Float)
    uv_index = Column(Float)
    magnitude = Column(Float)
    place = Column(String(80))
    lat = Column(String(12))
    long = Column(String(12))
    
# class LatLong(Base):
#     __tablename__ = 'latlong'
#     id = Column(Integer, primary_key=True)
#     lat = Column(String(12))
#     long = Column(String(12))
#     lat_rel = Column(Integer, ForeignKey('weatherSeries.id'))
#     weatherSer = relationship(WeatherSeries, primaryjoin=lat_rel == WeatherSeries.id)
#     latlong_id = Column(String,ForeignKey('latlong.id'))
#     latlong_rel = relationship(LatLong)
#     lat = Column(String(12))
#     long = Column(String(12))

In [5]:
# Create Database Connection
# ----------------------------------
# Creates a connection to our DB
# Engine opens the door. Conn is the walk through sign
engine = create_engine("sqlite:///data/py/earthquake_weather.sqlite")
conn = engine.connect()
# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine)
# Create a Session Object to Connect to DB
# ----------------------------------
session = Session(bind=engine)

In [ ]:
# ## Add weather series to sql
# x=0
# while x <= (len(all_logs) - 1):    
#     lat_long_data = LatLong(
#         lat = all_logs["lat"][x],
#         long = all_logs["long"][x]
#         )
#     x+=1
#     # Add Records to the Appropriate DB
#     # ----------------------------------
#     # Use the SQL ALchemy methods to run simple "INSERT" statements using the classes and objects  
#     session.add(lat_long_data)
#     # session.add(earthquake_data)
#     session.commit()
# print('Complete: Uploaded to SQLite DB')

In [ ]:
## Function: Add Weather Series to sql
def addToSQL(all_logs):
    x=0
    while x <= (len(all_logs) - 1):
        try: 
            weather_data = WeatherSeries(
                city = all_logs["city"][x],
                country = all_logs["country"][x],
                region = all_logs["region"][x],
                avgtemp = all_logs["avgtemp"][x],
                date = all_logs["date"][x],
                date_epoch = all_logs["date_epoch"][x],
                maxtemp = all_logs["maxtemp"][x],
                mintemp = all_logs["mintemp"][x],
                sunhour = all_logs["sunhour"][x],
                totalsnow = all_logs["totalsnow"][x],
                uv_index = all_logs["uv_index"][x],
                magnitude = all_logs["magnitude"][x],
                place = all_logs["place"][x],
                lat = all_logs["lat"][x],
                long = all_logs["long"][x],
                )
            x+=1
        except OperationalError:
            pass
            
        # Add Records to the Appropriate DB
        # ----------------------------------
        # Use the SQL ALchemy methods to run simple "INSERT" statements using the classes and objects  
        session.add(weather_data)
        # session.add(earthquake_data)
        session.commit()
    return 'Complete: Uploaded to SQLite DB'

In [ ]:
addToSQL(all_logs)

----

## Query from DB

DataFrame 

In [6]:
# Verify Data 
# Create DataFrame from sql table Weather
weather_data_df = pd.read_sql("SELECT * FROM WeatherSeries", conn)
weather_data_df.head()

,id,city,country,region,avgtemp,date,date_epoch,maxtemp,mintemp,sunhour,totalsnow,uv_index,magnitude,place,lat,long
0,1,Pasivaram,Tajikistan,Khatlon,75.0,2020-08-10,1.597018e+09,79.0,59.0,13.9,0.0,6.0,4.8,"19 km SE of Obigarm, Tajikistan",38.5804,69.8465
1,2,Pasivaram,Tajikistan,Khatlon,73.0,2020-08-11,1.597104e+09,79.0,59.0,13.9,0.0,6.0,4.8,"19 km SE of Obigarm, Tajikistan",38.5804,69.8465
2,3,Pasivaram,Tajikistan,Khatlon,73.0,2020-08-12,1.597190e+09,77.0,57.0,13.9,0.0,6.0,4.8,"19 km SE of Obigarm, Tajikistan",38.5804,69.8465
3,4,Pasivaram,Tajikistan,Khatlon,75.0,2020-08-13,1.597277e+09,79.0,59.0,13.8,0.0,6.0,4.8,"19 km SE of Obigarm, Tajikistan",38.5804,69.8465
4,5,Buuni Village,Tanzania,Pwani,79.0,2020-08-10,1.597018e+09,84.0,68.0,11.6,0.0,7.0,5.0,Tanzania,-7.3912,39.8924


In [7]:
weather_data_df[weather_data_df['lat'] == "53.42"]

,id,city,country,region,avgtemp,date,date_epoch,maxtemp,mintemp,sunhour,totalsnow,uv_index,magnitude,place,lat,long
8,9,Sanak,United States of America,Alaska,52.0,2020-08-09,1.596931e+09,54.0,52.0,10.4,0.0,3.0,5.7,"Unimak Island region, Alaska",53.42,-163.6973
9,10,Sanak,United States of America,Alaska,52.0,2020-08-10,1.597018e+09,52.0,43.0,10.4,0.0,3.0,5.7,"Unimak Island region, Alaska",53.42,-163.6973
10,11,Sanak,United States of America,Alaska,52.0,2020-08-11,1.597104e+09,52.0,45.0,9.4,0.0,3.0,5.7,"Unimak Island region, Alaska",53.42,-163.6973
11,12,Sanak,United States of America,Alaska,52.0,2020-08-12,1.597190e+09,52.0,36.0,9.4,0.0,3.0,5.7,"Unimak Island region, Alaska",53.42,-163.6973


#### Python call from db. For flask to web

In [ ]:
# # Python SQL toolkit and Object Relational Mapper
# import sqlalchemy
# # Go to existing database with automap_base
# from sqlalchemy.ext.automap import automap_base
# # Work through mapper to use python code
# from sqlalchemy.orm import Session
# # Inspect with python
# from sqlalchemy import create_engine, inspect


In [ ]:
# def weatherTimeSeries(query_call):
#     Base = automap_base()
#     Base.prepare(engine, reflect=True)
#     # Check db table names
#     # Base.classes.keys()
#     weather_table = Base.classes.weatherSeries
#     weather_container = session.query(weather_table).filter(weather_table.lat == query_call).all()
#     weather_data = []
#     for data in weather_container:
#         container = {
#             "city": data.city, 
#             "country": data.country, 
#             "region": data.region, 
#             "avgtemp": data.avgtemp, 
#             "date": data.date, 
#             "date_epoch": data.date_epoch, 
#             "maxtemp": data.maxtemp, 
#             "mintemp": data.mintemp, 
#             "sunhour": data.sunhour, 
#             "totalsnow": data.totalsnow, 
#             "uv_index": data.uv_index, 
#             "magnitude": data.magnitude, 
#             "place": data.place, 
#             "lat": data.lat, 
#             "long": data.long
#         }
#         weather_data.append(container)
#     return weather_data

In [ ]:
# latitude = '53.42'
# query_from_db_to_web = weatherTimeSeries(latitude)
# query_from_db_to_web

------

In [ ]:
# latlong_container = session.query(latlong_table).filter(latlong_table.lat == '').all()

In [ ]:
# for data in latlong_table:
#     print(f"Date: {data.lat}, Temp: {data.long}")

In [ ]:
# Join two tables in the db
# def join_example():
#     records = session.query(WeatherSeries).\
#         join(LatLong, LatLong.id == WeatherSeries.id).all()
# #     print(records)
#     for record in records:
#         recordObject = {
#             'city': record.city,
#             'country': record.country,
#             'date': record.date,
#             'maxtemp': record.maxtemp,
#             'latlong_rel': record.latlong_rel
#         }
#         print(recordObject)
# join_example()        

In [ ]:
# Create DataFrame from sql table Weather
# weather_data_df = pd.read_sql("SELECT * FROM LatLong", conn)
# weather_data_df

In [ ]:
### 
## KEEP !!
## DO NOT DELETE ALL THIS
###

# Create a Specific Instance of the "weather_data" and "earthquake_data" classes
# ----------------------------------
# Data needs to pass through this. A for loop while need to be created to consistantly send multiple files
# weather_data = WeatherSeries(
#     lat = '',
#     lon = '',
#     date = '',
#     city = '',
#     country = '',
#     region = '',
#     avgtemp = '',
#     date_epoch = '',
#     maxtemp = '',
#     mintemp = '',
#     sunhour = '',
#     totalsnow = '',
#     uv_index = '',
#     )

# earthquake_data = Earthquake(
#     lat = '',
#     lon = '',
#     date = '',
#     mag = '',
#     location = ''
#     )

-------------

-----------

In [ ]:
# def checkEqual3(lst):
#     return lst[1:] == lst[:-1]

# def checkRowValueDups(data):
#     dictOfKeys = data.keys()
#     for column in data:
#         column_values = data[column].to_list()
#         result = checkEqual3(column_values)
# #         print('passed.')
#     if result == True:
#         all_logs_df = all_logs.iloc[0:1, 0:]
#     else:
#         print("Couldnt complete cleaning. Different value found in column where only duplicates live. Check function checkRowValueDups() for information")
#     return all_logs_df

# all_logs_df = checkRowValueDups(all_logs)

In [ ]:
# all_logs_df['2020-08-03'][0]

In [ ]:
# stuff = (list(all_logs.itertuples(index=False, name=None)))